# Нефтедобыча

---

# Описание проекта

Даны пробы нефти в трёх регионах. Требуется построить модель машинного обучения, которая поможет определить регион с наиболее прибыльной добычей.

# Описание данных
Данные геологоразведки трёх регионов находятся в файлах: geo_data_0.csv, geo_data_1.csv, geo_data_2.csv.
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек;
* product — объём запасов в скважине (тыс. баррелей).

# План действий
1. Загрузить и подготовить данные. Пояснить порядок действий.

2. Обучить и проверить модель для каждого региона:

    2.1. Разбить данные на обучающую и валидационную выборки в соотношении 75:25.
    
    2.2. Обучить модель и сделать предсказания на валидационной выборке.
    
    2.3. Сохранить предсказания и правильные ответы на валидационной выборке.
    
    2.4. Напечатать на экране средний запас предсказанного сырья и RMSE модели.
    
    2.5. Проанализировать результаты.
    
    
3. Подготовиться к расчёту прибыли:

    3.1. Все ключевые значения для расчётов сохранить в отдельных переменных.
    
    3.2. Рассчитать достаточный объём сырья для безубыточной разработки новой скважины. Сравнить полученный объём сырья со средним запасом в каждом регионе.
    
    3.3. Написать выводы по этапу подготовки расчёта прибыли.
    
    
4. Написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

    4.1. Выбрать скважины с максимальными значениями предсказаний.
    
    4.2. Просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям.
    
    4.3. Рассчитать прибыль для полученного объёма сырья.
    
    
5. Посчитать риски и прибыль для каждого региона:

    5.1. Применить технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    
    5.2. Найти среднюю прибыль, 95%-й доверительный интервал и риск убытков.
    
    5.3. Написать выводы: предложить регион для разработки скважин и обосновать выбор.

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
data_reg_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_reg_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_reg_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_reg_0.sort_values(by='product',ascending = False).head()

,id,f0,f1,f2,product
8826,rjMou,1.797736,0.098212,6.148260,185.364347
99818,7cHIv,0.518445,-0.412790,4.951916,185.362690
94175,uCDzR,0.351428,-0.400244,7.227618,185.355615
1925,IfqrC,0.624430,-0.469312,5.753677,185.354980
45291,5FEPb,1.758787,-0.395038,6.719085,185.352015


In [4]:
data_reg_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_reg_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [6]:
data_reg_0.duplicated().sum()

0

In [7]:
data_reg_1.sort_values(by='product',ascending = False).head()

,id,f0,f1,f2,product
53864,MzRzn,2.901352,-3.475398,5.001393,137.945408
97083,nW6eC,-4.030430,-14.020643,5.009571,137.945408
88340,xwJzQ,-3.373117,-9.227661,4.994369,137.945408
64879,Xd8DC,-5.360281,-2.388204,5.001439,137.945408
7288,Stdrb,-8.058661,-2.244840,5.000753,137.945408


In [8]:
data_reg_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_reg_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
data_reg_1.duplicated().sum()

0

In [11]:
data_reg_2.sort_values(by='product',ascending = False).head()

,id,f0,f1,f2,product
79705,UAhji,-2.747914,1.555227,3.342182,190.029838
93444,IB0JE,3.026506,1.344623,8.891243,190.013589
35099,2HeCn,2.848844,-1.701996,9.437101,190.011722
21943,dldNH,-2.500091,2.024392,6.291513,190.010982
37870,CpuBi,-0.686667,-3.219452,5.963290,190.010029


In [12]:
data_reg_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
data_reg_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [14]:
data_reg_2.duplicated().sum()

0

In [15]:
# Столбец с идентификаторами на всякий случай сохраним в отдельной таблице
data_id_0 = data_reg_0['id']
data_id_1 = data_reg_1['id']
data_id_2 = data_reg_2['id']

In [16]:
# Удаляем ненужный столбец id из датасетов (можно объединить с действиями в следующей ячейке, но для прозрачности оставим отдельно)
data_reg_0 = data_reg_0.drop('id', axis=1)
data_reg_1 = data_reg_1.drop('id', axis=1)
data_reg_2 = data_reg_2.drop('id', axis=1)

In [17]:
# Разделяем признаки и целевой признак
target_0 = data_reg_0['product']
features_0 = data_reg_0.drop('product', axis=1)

target_1 = data_reg_1['product']
features_1 = data_reg_1.drop('product', axis=1)

target_2 = data_reg_2['product']
features_2 = data_reg_2.drop('product', axis=1)

## Выводы по шагу 1
Что сделано:
1. Загрузили 3 датасета по 3 регионам, в каждом 100 000 объектов, дубликатов и пропусков нет.
2. Столбцы с идентификаторами скважин вынесли в отдельные датафреймы и удалили из основных датафреймов.
3. Разделили в датасетах признаки (f0,f1,f2) и целевой признак (product).

# 2. Обучение и проверка модели

In [18]:
# Разбиваем датасетs на выборки (обучающую, проверочную)
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

In [19]:
# Функция обучения и предсказаний модели, вычисления RMSE
def mse(features_train,target_train,features_valid,target_valid,reg):
    model = LinearRegression()
    model.fit(features_train, target_train)
    pred_valid_ = model.predict(features_valid)
    pred_valid_ = pd.Series(pred_valid_, index=features_valid.index)
    mse_ = mean_squared_error(target_valid, pred_valid_)

    print('РЕГИОН',reg)
    print("R2=",model.score(features_valid, target_valid))
    print("MSE =", mse_)
    print("RMSE =", mse_ ** 0.5)
    print("Средний запас предсказанного сырья в регионе, тыс. баррелей =", round(pred_valid_.mean(),2),"(по факту=",round(target_valid.mean(),2),")\n")
    return pred_valid_ 


In [20]:
pred_valid_0 = mse(features_train_0,target_train_0,features_valid_0,target_valid_0,0)
pred_valid_1 = mse(features_train_1,target_train_1,features_valid_1,target_valid_1,1)
pred_valid_2 = mse(features_train_2,target_train_2,features_valid_2,target_valid_2,2)

РЕГИОН 0
R2= 0.27994321524487786
MSE = 1412.2129364399243
RMSE = 37.5794217150813
Средний запас предсказанного сырья в регионе, тыс. баррелей = 92.59 (по факту= 92.08 )

РЕГИОН 1
R2= 0.9996233978805126
MSE = 0.7976263360391139
RMSE = 0.893099286775616
Средний запас предсказанного сырья в регионе, тыс. баррелей = 68.73 (по факту= 68.72 )

РЕГИОН 2
R2= 0.20524758386040443
MSE = 1602.3775813236196
RMSE = 40.02970873393434
Средний запас предсказанного сырья в регионе, тыс. баррелей = 94.97 (по факту= 94.88 )



In [21]:
# Посмотрим, что там в предсказаниях
pred_valid_0

71751     95.894952
80493     77.572583
2655      77.892640
53233     90.175134
91141     70.510088
            ...    
12581    103.037104
18456     85.403255
73035     61.509833
63834    118.180397
43558    118.169392
Length: 25000, dtype: float64

## Выводы по шагу 2
Что сделано:
В каждом регионе:
1. Разбили данные на обучающую и валидационную выборки в соотношении 75:25.
2. Обучили модель линейной регрессией и сделали предсказания на валидационной выборке.
3. Сохранили предсказания и правильные ответы на валидационной выборке.
4. Вывели на экран средний запас предсказанного сырья и RMSE модели.

Судя по RMSE и R2 лучше всего обучилась модель по датасету региона 1.
При этом в регионе 1 средний запас предсказанного сырья на треть меньше чем в регионах 0 и 2

# 3. Подготовка к расчёту прибыли

In [22]:
# Ключевые значения для расчетов

budget_region = 10000000000 #бюджет на разработку региона
research_points = 500 #количество исследуемых в регионе точек
best_points = 200 #количество отбираемых точек
barrel_cost = 450000 #стоимость 1000 баррелей (1 единицы продукта) в рублях
loss = 0.025 #максимальная вероятность убытка
boot_size = 1000 #количество выборок для bootstrap

budget_point = budget_region/best_points #бюджет на одну скважину
print('Бюджет на одну скважину, руб.:',budget_point)
min_product = round(budget_point/barrel_cost,6) #минимальный объем сырья в скважине для безубыточной разработки (в тысячах баррелей)
print('\nМинимальный объем сырья в скважине для безубыточной разработки (тыс. баррелей):',min_product)

Бюджет на одну скважину, руб.: 50000000.0

Минимальный объем сырья в скважине для безубыточной разработки (тыс. баррелей): 111.111111


## Выводы по шагу 3
Что сделано:
1. Сохранили ключевые значения в отдельных переменных:

    *budget_region = 10000000000* - бюджет на разработку региона    
    *research_points = 500* - количество исследуемых в регионе точек    
    *best_points = 200* - количество отбираемых точек    
    *barrel_cost = 450000* - стоимость 1000 баррелей (1 единицы продукта) в рублях    
    *loss = 0.025* - максимальная вероятность убытка    
    *boot_size = 1000* - количество выборок для bootstrap
    
    
2. Посчитали бюджет на одну скважину, исходя из бюджета на регион (10 млрд. руб) и количества разрабатываемых скважин (200) = 50 млн. руб.


3. Рассчитали достаточный объём сырья для безубыточной (чтобы хотя бы выйти в ноль) разработки новой скважины: 111.111111 тыс. баррелей. 

Средний запас предсказанного сырья в каждом регионе меньше объема, достаточного для безубыточной разработки скважины - т.е., не все скважины будут безубыточны.


# 4. Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [23]:
# Рандомайзер
state = np.random.RandomState(12345)

In [24]:
def Profit(target_valid,pred_valid,reg,out):
    # Берем 500 случайных скважин из предсказаний
    random_research_pred = pred_valid.sample(research_points, random_state=state)
    # Из них берем 200 лучших по предсказаниям
    best_research_pred = random_research_pred.sort_values(ascending = False)[:best_points]
    # Считаем суммарный объем сырья по валидационной выборке
    best_product_sum_target = target_valid[best_research_pred.index].sum()
    # Считаем прибыль от 200 скважин (выручка минус стоимость разработки)
    profit = best_product_sum_target*barrel_cost - budget_region
    if out == 1:
        print('Суммарный объем сырья в 200 лучших скважинах из валидационной выборки, тыс. баррелей:',round(best_product_sum_target))
        print('Прибыль от 200 лучших скважин в регионе',reg,', млн. руб:',round(profit/1000000),'\n')
    return profit

In [25]:
profit_0 = Profit(target_valid_0,pred_valid_0,0,1)
profit_1 = Profit(target_valid_1,pred_valid_1,1,1)
profit_2 = Profit(target_valid_2,pred_valid_2,2,1)

Суммарный объем сырья в 200 лучших скважинах из валидационной выборки, тыс. баррелей: 23731.0
Прибыль от 200 лучших скважин в регионе 0 , млн. руб: 679.0 

Суммарный объем сырья в 200 лучших скважинах из валидационной выборки, тыс. баррелей: 22784.0
Прибыль от 200 лучших скважин в регионе 1 , млн. руб: 253.0 

Суммарный объем сырья в 200 лучших скважинах из валидационной выборки, тыс. баррелей: 23315.0
Прибыль от 200 лучших скважин в регионе 2 , млн. руб: 492.0 



In [26]:
reg_profit = pd.Series([profit_0,profit_1,profit_2])
best_reg = reg_profit.sort_values(ascending = False).index[0]
print('Лучший по прибыли регион:',best_reg)

Лучший по прибыли регион: 0


## Выводы по шагу 4
Что сделано: 
По каждому региону:
1. Из 25000 предсказаний взяли 500 случайных.
2. Из 500 случайных предсказаний выбрали 200 лучших по объему сырья скважин
3. Просуммировали объем сырья по правильным ответам с индексами отобранных 200 лучших предсказаний
4. Рассчитали прибыль для полученного объёма сырья. В данном случае поскольку каждый раз 500 скважин для исследования берутся случайным образом, то и результат (прибыль от 200 лучших скважин в регионе) каждый раз получается разным.

# 5. Расчёт прибыли и рисков 
Переменные, которые мы ранее определили:

*budget_region = 10000000000* - бюджет на разработку региона

*research_points = 500* - количество исследуемых в регионе точек

*best_points = 200* - количество отбираемых точек

*barrel_cost = 450000* - стоимость 1000 баррелей (1 единицы продукта) в рублях

*loss = 0.025* - максимальная вероятность убытка

*boot_size = 1000* - количество выборок для bootstrap

In [27]:
# Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.# Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
def Bootstrap_Profit(target_valid,pred_valid,reg):
    values = []
    loss_reg = 0
    for i in range(boot_size): #boot_size = 1000
        
        profit = Profit(target_valid,pred_valid,reg,0)
        
        values.append(profit)
        
        #если прибыль отрицательная, считаем за убыток
        if profit < 0:
            loss_reg += 1
            
    values = pd.Series(values)

    print('РЕГИОН',reg)

    profit_mean = values.mean()
    print('Средняя прибыль, млн. руб:',round(profit_mean/1000000))
    
    lower = values.quantile(0.025)
    upper =values.quantile(0.975) 
    print('95% доверительный интервал:',round(lower/1000000),'-',round(upper/1000000))
    proba_loss = loss_reg/boot_size #boot_size = 1000
    print('Вероятность убытков:',proba_loss)
    if proba_loss < loss: #loss = 0.025
        print('Вероятность убытков меньше',loss,'\n')
    else:
        print('Вероятность убытков не меньше',loss,'\n')    

    return profit_mean,proba_loss


In [28]:
# Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков
profit_mean_0,proba_loss_0 = Bootstrap_Profit(target_valid_0,pred_valid_0,0)
profit_mean_1,proba_loss_1 = Bootstrap_Profit(target_valid_1,pred_valid_1,1)
profit_mean_2,proba_loss_2 = Bootstrap_Profit(target_valid_1,pred_valid_2,2)

РЕГИОН 0
Средняя прибыль, млн. руб: 381.0
95% доверительный интервал: -127.0 - 880.0
Вероятность убытков: 0.072
Вероятность убытков не меньше 0.025 

РЕГИОН 1
Средняя прибыль, млн. руб: 455.0
95% доверительный интервал: 47.0 - 840.0
Вероятность убытков: 0.013
Вероятность убытков меньше 0.025 

РЕГИОН 2
Средняя прибыль, млн. руб: -3824.0
95% доверительный интервал: -4415.0 - -3247.0
Вероятность убытков: 1.0
Вероятность убытков не меньше 0.025 



In [29]:
# После оценки рисков оставим лишь те регионы, в которых вероятность убытков меньше 2.5%, среди них выберем регион с наибольшей средней прибылью.
data_reg_profit_loss = pd.DataFrame({'reg': [0,1,2], 'profit_mean': [profit_mean_0,profit_mean_1,profit_mean_2], 'proba_loss': [proba_loss_0,proba_loss_1,proba_loss_2]})

best_of_the_best = int(data_reg_profit_loss.query('proba_loss < @loss').sort_values(by='profit_mean',ascending = False).iloc[0]['reg'])
print('Регион, где можно получить максимальную прибыль:',best_of_the_best)

Регион, где можно получить максимальную прибыль: 1


## Выводы по шагу 5
Что сделано:
Для каждого региона:
1. Применили на правильных ответах технику Bootstrap с 1000 выборок.
2. Нашли среднюю прибыль, 95%-й доверительный интервал и риск убытков.

###### Лучший регион для разработки скважин: 1. Здесь максимальная средняя прибыль и при этом риск убытков меньше 2.5%.